# Task 1: Implement a Quantum Oracle

*In this task, you should implement your solution in Q# in the first cell of the notebook. The rest of the cells contain the testing harness used to verify your solution. You can use them when working on your solution, but any modifications to them will not be evaluated.*

*To submit this task, update this notebook with your solution code in the first cell and commit it in your challenge repository. You don't need to modify anything else in the notebook, or save cell execution results (unlike in the later tasks!)*

### <span style="color:blue">Task 1</span>: Implement a quantum oracle

**Inputs:**

  1. $N$ qubits in an arbitrary state $|x\rangle$ (input/query register)
    
  2. A qubit in an arbitrary state $|y\rangle$ (target qubit)

**Goal:**

Implement a marking oracle for function $f(x) = 1\text{ if x modulo 4 is 0 or 3}$.

That is, if both inputs are in a basis state, flip the state of the output qubit 
if and only if the number written in the array "x" modulo 4 equals 0 or 3,
and leave the state of the array "inputs" unchanged.
The effect of the oracle on superposition states should be defined by its linearity.

The array "inputs" uses little-endian encoding, i.e., the least significant bit of the integer is stored first.

Don't use measurements; the implementation should use only X and controlled X gates.
This task will be tested using ToffoliSimulator.

**Example:**

The result of applying the oracle to a state 
$$\frac1{\sqrt3}\big(|001\rangle + |100\rangle + |111\rangle\big) \otimes |0\rangle$$
will be 
$$\frac1{\sqrt3}\big(|001\rangle \otimes |1\rangle + |100\rangle \otimes |0\rangle + |111\rangle \otimes |1\rangle\big)$$

In [ ]:
operation Task1 (x : Qubit[], y : Qubit) : Unit is Adj + Ctl {
    // ...
}

### <span style="color:blue">Testing Harness</span>

*To test your solution, execute the second and the third code cells. The third cell will produce the test result - a message "Success!" if your solution is correct, or an error message explaining the test case on which it fails.*

*You need to re-run all the cells every time you update your solution - otherwise you'll be testing the old version of your solution!*

In [ ]:
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation VerifySingleOutputFunction(
    numInputs : Int, 
    op : (Qubit[], Qubit) => Unit is Adj+Ctl, 
    predicate : Int -> Bool
) : Unit {
    for assignment in 0 .. 2^numInputs - 1 {
        use (inputs, output) = (Qubit[numInputs], Qubit());
        within {
            ApplyXorInPlace(assignment, LittleEndian(inputs));
            AllowAtMostNCallsCA(0, Measure, "You are not allowed to use measurements");
        } apply {
            op(inputs, output);
        }

        // Check that the result is expected
        let actual = ResultAsBool(MResetZ(output));
        let expected = predicate(assignment);
        Fact(actual == expected,
            $"Oracle evaluation result {actual} does not match expected {expected} for assignment {IntAsBoolArray(assignment, numInputs)}");

        // Check that the inputs were not modified
        Fact(MeasureInteger(LittleEndian(inputs)) == 0, 
            $"The input states were modified for assignment {assignment}");
    }
}


// ------------------------------------------------------
function Is03ModFour (input : Int) : Bool {
    return input % 4 == 0 or input % 4 == 3;
}

// ------------------------------------------------------
operation TestIs03ModFour() : Unit {
    for i in 2 .. 5 {
        VerifySingleOutputFunction(i, Task1, Is03ModFour);
    }
    Message("Success!");
}


In [ ]:
%toffoli TestIs03ModFour